In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from librosa import note_to_hz
from scipy.io import wavfile
from IPython.display import display, Audio
from matplotlib import pyplot

sys.path.append('../tuner')
from tuner_api_wrapper import *

In [ ]:
WINDOW_SIZE = 512
OSAMP = 64
DO_HIGHPASS = False
DO_HILBERT_TRANSFORM = False
SMOOTHING_WINDOW_1 = 1
SMOOTHING_WINDOW_2 = 1

In [ ]:
current_dir = str(Path().resolve())
audio_path = os.path.normpath(current_dir + '/../audio/Lopez Island.wav')
sample_rate, audio = wavfile.read(audio_path)

In [ ]:
detected_freqs = tuner_pitch_detect(
    audio, 
    sample_rate,
    WINDOW_SIZE,
    OSAMP,
    DO_HIGHPASS,
    DO_HILBERT_TRANSFORM,
)

In [ ]:
detected_smoothed_freqs = tuner_smooth(detected_freqs, SMOOTHING_WINDOW_1)

In [ ]:
c_major = ['C#', 'D', 'E', 'F#', 'G#', 'A', 'B']
min_octave = -8
max_octave = 8
scale = []
for octave in range(min_octave, max_octave):
    for note in c_major:
        scale.append(note_to_hz(note + str(octave)))

In [ ]:
corrected_freqs = tuner_pitch_snap(detected_freqs, scale)

In [ ]:
corrected_smoothed_freqs = tuner_smooth(corrected_freqs, SMOOTHING_WINDOW_2)

In [ ]:
shifted_audio = tuner_pitch_shift(
    audio,
    sample_rate,
    WINDOW_SIZE,
    OSAMP,
    detected_freqs,
    corrected_smoothed_freqs,
)

In [ ]:
print('Detected frequencies')
pyplot.plot(detected_freqs)
pyplot.show()
print('Detected frequencies (smoothed)')
pyplot.plot(detected_smoothed_freqs)
pyplot.show()
print('Corrected frequencies')
pyplot.plot(corrected_freqs)
pyplot.show()
print('Corrected frequencies (smoothed)')
pyplot.plot(corrected_smoothed_freqs)
pyplot.show()
print('Original audio')
display(Audio(audio, rate=sample_rate))
print('Pitch shifted audio')
display(Audio(shifted_audio, rate=sample_rate))